In [1]:
from g4f.client import Client
import re
import pandas as pd

In [2]:
client = Client()

In [3]:
def generate_csv_report(user_input, csv_file_path):
    """
    參考CSV檔案中的數據，並進行分析。
    Args:
        user_input (str): 用戶的自然語言查詢。
        csv_file_path (str): CSV 檔案的路徑。

    """

    # 讀取 CSV 檔案並將內容轉換為字串
    try:
        df = pd.read_csv(csv_file_path)
        csv_string = df.to_string(index=False)  # 不顯示索引
    except FileNotFoundError:
        print("找不到 CSV 檔案！請檢查檔案路徑是否正確。")
        return ""

    # 構建系統提示和用戶提示
    system_prompt = """
    1. 你是財經數據分析專家，能夠分析csv檔中的指標數據，並且給出詳盡的分析報告。
    2. 可以上網搜尋相對應的閥值來判斷是否有風險。
    3. 可以提出指標後續趨勢判斷。
    """

    user_prompt = f"""
    1.參考此檔案並且給出分析報告:{csv_string}
    """

    # 使用 OpenAI API 生成報告
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input + user_prompt}
        ],
        temperature=0.1
    )
    text = response.choices[0].message.content
    return text


In [4]:
user_query ="此為某公司財務指標數據，請幫忙分析此數據並提出財務建議!"
csv_file = ".\csv.ingredient\指標管制.csv"
text = generate_csv_report(user_query,csv_file)
print(text)

New g4f version: 0.4.0.0 (current: 0.3.9.7) | pip install -U g4f
根據您提供的財務指標數據，我將進行分析並提出相應的財務建議。以下是對數據的詳細分析報告：

### 數據概覽
- 數據範圍：2020年1月至2021年12月
- 指標：p值（可能代表某種財務比率或指標的百分比）

### 數據分析
1. **趨勢分析**：
   - 整體來看，p值在2020年和2021年之間有明顯的波動。
   - 2020年1月的p值為10.00%，隨後在2月和3月出現下降，最低點為2020年4月的0.00%。
   - 2020年5月至2020年12月，p值逐漸回升，並在2021年達到最高點11.24%（2021年7月）。

2. **標準差分析**：
   - 數據中提到的σ線（標準差）指標顯示，某些月份的p值超過了上2σ和上3σ線，這可能表示這些月份的財務表現異常。
   - 例如，2020年4月的p值為13.00%，超過了上3σ線，這可能是異常的高值，需進一步調查原因。

3. **平均值計算**：
   - 整體平均值為5.33%，這意味著大多數月份的p值低於此平均水平，顯示出一定的波動性。

### 風險評估
- **高風險月份**：
  - 2020年4月的p值異常高，可能需要進一步分析該月的財務狀況及外部因素。
  - 2021年7月的p值也達到11.24%，需評估是否持續增長或回落。

- **低風險月份**：
  - 2020年4月的0.00%和2021年8月的1.65%顯示出潛在的低風險，但也可能反映出業務活動的減少。

### 財務建議
1. **持續監控**：
   - 建議定期監控p值的變化，特別是在異常高或低的月份，並分析背後的原因。

2. **風險管理**：
   - 對於高風險月份，應該進行深入的財務分析，了解是否存在潛在的風險因素，例如市場變化、成本上升或需求下降。

3. **預算調整**：
   - 根據過去的數據趨勢，考慮調整未來的預算和資金分配，以應對可能的波動。

4. **增強財務彈性**：
   - 建議建立應急預算，以應對未來可能出現的財務不穩定情況。

### 結論
整體而言，該公司的財務指標顯示出一定的波動性，需加強對異常情況的監控和分析。通過持續的數據分析和風